# Machine Learning with Apache Spark

In [1]:
import findspark
findspark.init()

## Introduction

Spark is a framework for working with Big Data. In this chapter you'll cover some background about SPark and Machine Learning. You'l then find out how to connect to Spark using Python and load CSV data.

### Machine Learning & Spark

Data in RAM
- The performance on a ML model depends on data
- If the data can't fit entirely in RAM then it will start to page data between RAM and Disk and the bigger the data the more time the computer spends waiting for data.
- One option to solve is to distribute the data amoung clusters.

### Characteristics of Spark

Spark is currently the most popular technology for processing large quantities of data. Not only is it able to handle enormous data volumes, but it does so very efficiently too! Also, unlike some other distributed computing technologies, developing with Spark is a pleasure.

### Components in a Spark Cluster
Spark is a distributed computing platform. It achieves efficiency by distributing data and computation across a cluster of computers.
A Spark cluster consists of a number of hardware and software components which work together.

### Connecting to Spark
Languages for interacting with Spark.
- Java - low-level, compiled
- Scala, Python, and R - high-level with interactive REPL (read.eval.print.loop)

Importing pyspark
- import `pyspark`

Sub-modules
- Structured Data - `pyspark.sql`
- Streaming Data - `pyspark.streaming`
- Machine Learning - `pyspark.mllib` (deprecated) and `pyspark.ml`

Spark URL
- **Remote Cluster** using Spark URL - spark://<IP address | DNS name>:<\port>
    - *example* spark://13.59.151.161:7077
- **Local Cluster**
    - `local` - only 1 core;
    - ` local[4]` - 4 cores;
    - `locall[*]` - all available cores.

Creating a SparkSession
- connect to spark by using:
<br>```from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('first_spark_app').getOrCreate()```

- close connection to spark
<br>`spark.stop()`

### Location of Spark master
The following are all examples of valid ways to specify the location of a Spark Cluster:
- spark://13.59.151.161:7077
- spark://ec2-18-188-22-23.us-east-2.compute.amazonaws.com:7077
- local
- local[4]
- local[*]

### Creating a SparkSession

In [2]:
# import the PySpark module
from pyspark.sql import SparkSession

# create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('test') \
                    .getOrCreate()

# what version of Spark?
print(spark.version)

2.4.3


### Loading Data

`DataFrame` for tabular data.

Reading data from CSV
<br>&emsp;`cars = spark.read.csv('cars.csv', header=True)`

- Optional arguments:
    - `header` - is first row a header? (defaule:`False`)
    - `sep` - field separator (default: a comma `','`)
    - `schema` - explicit column data types
    - `inferSchema` - deduce column data types from data?
    - `nullValue` - placeholder for missing data
    
Peek at the data
<br>&emsp;&emsp;`cars.show(5)`

### Loading flights data

In [3]:
# read data from CSV file
flights = spark.read.csv('../data/flights.csv',
                        sep=',',
                        header=True,
                        inferSchema=True,
                        nullValue='NA')

# get number of records
print("The data contain %d records." % flights.count())

# view the first five records
flights.show(5)

# check column data types
flights.dtypes

The data contain 50000 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65| null|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



[('mon', 'int'),
 ('dom', 'int'),
 ('dow', 'int'),
 ('carrier', 'string'),
 ('flight', 'int'),
 ('org', 'string'),
 ('mile', 'int'),
 ('depart', 'double'),
 ('duration', 'int'),
 ('delay', 'int')]

### Loading SMS spam data

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# specify column names and types
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])

# load data from a delimited file
sms = spark.read.csv("../data/sms.csv", sep=';', header=False, schema=schema)

# print schema of DataFrame
sms.printSchema()

root
 |-- id: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- label: integer (nullable = true)



## Classification
Now that you are familiar with getting data into Spark, you'll move onto buiilding two types of classification model: Decision Trees and Logistic Regression. You'll also find out about a few approaches to data preparation.

### Data Preparation
Dropping Colunns
- Drop the columns you don't want
<br>&emsp;`cars = cars.drop('maker', 'model')`

- Select columns you want to retain
<br>&emsp;`cars = cars.select('origin', 'type', 'cyl', 'size', 'weight', 'length', 'rpm', 'consumption')`

Filtering out missing data
- How many missing values?
<br>&emsp;`cars.filter('cyl IS NULL').count()`

- Drop records with missing values in the `cylinders` column
<br>&emsp;`cars = cars.filter('cyl IS NOT NULL')`

- Drop records with missing values in **any** column
<br>&emsp;`cars = cars.dropna()`

Mutating columns
<br>&emsp;`from pyspark.sql.functions import round`
<br>&emsp;
<br>&emsp;`cars = cars.withColumn('mass', round(cars.weight / 2.205, 0)`
<br>&emsp;
<br>&emsp;`cars.withColumn('length', round(cars.length * 0.0254, 3))`

Indexing categorical data
<br>&emsp;`From pyspark.ml.feature import StringIndexer`
<br>&emsp;
<br>&emsp; `indexer = StringIndexer(inputCol='type', outputCol='type_idx')`
<br>&emsp;
<br>&emsp;`indexer = indexer.fit(cars)`
<br>&emsp;
<br>&emsp;`cars = indexer.transform(cars)`

Assembling columns
- consolidate input columns into a single column
- ML algorithms in Spark operate on a single vector
<br>&emsp;`from pyspark.ml.feature import VectorAssembler`
<br>&emsp;
<br>&emsp;`assembler = VectorAssembler(inputCols=['cyl','size'], outputCol='features')`
<br>&emsp;`assembler.transform(cars)`

### Removing columns and rows

In [5]:
# remove the 'flight' column
flights = flights.drop('flight')

# number of records with missing 'delay' values
flights.filter('delay IS NULL').count()

# remove records with missing 'delay' values
flights = flights.filter('delay IS NOT NULL')

# remove records with missing values in any column and get the number of remaining rows
flights = flights.dropna()
print(flights.count())

47022


### Column manipulation

In [6]:
# import the required function
from pyspark.sql.functions import round

# convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934)).drop('mile')

# create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# check first five records
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



### Categorical columns

In [7]:
from pyspark.ml.feature import StringIndexer

# create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# indexer identifies catgories in the data
indexer_model = indexer.fit(flights_km)

# indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

### Assembling columns

In [8]:
# import the necessary class
from pyspark.ml.feature import VectorAssembler

# create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration'
], outputCol='features')

# consolidate predictor columns
flights_assembled = assembler.transform(flights_indexed)

# check the resulting columns
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30   |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8   |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5   |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2    |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54   |
+-----------------------------------------+-----+
only showing top 5 rows



### Decision Tree

Recursive partitioning

Stopping criteria examples (stop splitting)
- number of records of a node fall below a threshold
- purity of a node is above a threshold

Split train/test
- `train, test = cars.randomSplit([0.8, 0.2], seed=23)`

Build a decision tree
- `from pyspark.ml.classification import DecisionTreeClassifier`
- `tree = DecisionTreeClassifier()`
- `tree_model = tree.fit(train)`
- `prediction = tree.transform(test)`

Confusion matrix
- A confusion matrix is a table which describes performance of a model on testing data.
- `prediction.groupBy("label", "prediction").count().show()`
- *note Accuracy = (TN + TP)/(TN + TP + FN + FP) - proportion of correct predictions*

### Train/test split

In [9]:
# split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# check that training set has around 80% of recods
training_ratio = flights_train.count()/flights.count()
print(training_ratio)

0.7980732423121092


### Build a Decision Tree

In [10]:
# import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+---------------------------------------+
|label|prediction|probability                            |
+-----+----------+---------------------------------------+
|1    |1.0       |[0.493801652892562,0.506198347107438]  |
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
|1    |1.0       |[0.3550259700580507,0.6449740299419493]|
+-----+----------+---------------------------------------+
only showing top 5 rows



In [11]:
flights_assembled.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|            features|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|[0.0,22.0,2.0,0.0...|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|[2.0,20.0,4.0,0.0...|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|        1.0|    0.0|[9.0,13.0,1.0,1.0...|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|        0.0|    1.0|[5.0,2.0,1.0,0.0,...|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|        1.0|    0.0|[7.0,2.0,6.0,1.0,...|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+--------------------+
only showing top 5 rows



### Evaluate the Decision Tree

In [12]:
# create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# accuracy measure the propeortion of correct predictions
accuracy = (TN + TP)/(TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1218|
|    0|       0.0| 2424|
|    1|       1.0| 3606|
|    0|       1.0| 2247|
+-----+----------+-----+

0.6350710900473934


### Logistic Regression

- Uses a logistic function to measure a binary target (0,1 : true, false)
- X-axis is a linear combination of predictor variables
- y-axis output of the model
- the model dreives coefficients for each numerical predictor
    - can shift the curve to the right or left
    - can make the transition more gradual or steep
    
Build a Logistic Regression model
- `from pyspark.ml.classification import LogisticRegression`
- `logistic = LogisticRegression()`
- `logistic_model = logistic.fit(train)`
- `prediction = logistic_model.transform(test)`

Precision and recall
- build confusion matrix
- precision = TP / (TP + FP) *note proportion of positive predictions*
- recall = TP / (TP + FN) *note proportion of positive targets that are correctly predicted*

Weighted metrics
- `from pyspark.ml.evaluation import MulticlassClassificationEvaluator`
- `evaluator = MulticlassClassificationEvaluator()`
- `evaluator.evaluate(prediction, {evaluator.metricName: 'weightedPrecision'})`
    - Other metrics: `weightedRecall`, `accuracy`, `f1`
    
ROC and AUC
ROC = "Receiver Operating Characteristic"
- TP versus FP
- threshold = 0 (top right)
- threshdol = 1 (bottom left)
AUC = "Area under the curve"
- ideally AUC = 1

### Build a Logistic Regression model

In [13]:
flights_num = flights_assembled.select('mon', 'depart', 'duration', 'features', 'label')
flights_num_train, flights_num_test = flights_num.randomSplit([0.8, 0.2], seed=17)

# import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# create a classifier object and train on the training data
logistic = LogisticRegression().fit(flights_num_train)

# create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_num_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1666|
|    0|       0.0| 2636|
|    1|       1.0| 3169|
|    0|       1.0| 2024|
+-----+----------+-----+



### Evaluate the Logistic Regression model

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall = {:.2f}'.format(precision, recall))

# find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction,
                                              {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.61
recall = 0.66


### Turning Text into Tables

Removing punctuation
- `from pyspark.sql.functions import regexp_replace`
- `REGEX = '[,\\-]'`
- `books = books.withColumn('text', regexp_replace(books.text, REGEX, ' '))`

Text to tokens
- `from pyspark.ml.feature import Tokenizer`
- `books = Tokenizer(inputCol="text", outputCol="tokens").transform(books)`

What are stop words"
- `from pyspark.ml.feature import StopWordsRemover`
- `stopwords = StopWordsRemover()`
- `stopwords.getStopWords()`

Removing stop words
- `stopwords = stopwords.setInputCol('tokens').setOutputCol('words')`
- `books = stopwords.transform(books)`

Feature hashing
- `from pyspark.ml.feature import HashingTF`
- `hasher = HashingTF(inputCol="words", outputCol="hash", numFeatures=32)`
- `books = hasher.transform(books)`

Dealing with common words
- IDF = inverse documents frequency
- `from pyspark.ml feature import IDF`
- `books = IDF(inputCol="hash", outputCol="features").fit(books).transform(books)`

### Punctuation, numbers and tokens

In [15]:
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# remove punctuation
sms = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
sms = sms.withColumn('text', regexp_replace(sms.text, '[0-9]', ' '))

# merge multiple spaces
sms = sms.withColumn('text', regexp_replace(sms.text, ' +', ' '))

# split the text into words
sms = Tokenizer(inputCol='text', outputCol='words').transform(sms)

sms.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



### Stop words and hashing


In [16]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# remove stop words
sms = StopWordsRemover(inputCol="words", outputCol="terms").transform(sms)

# apply the hashing trick
sms = HashingTF(inputCol="terms", outputCol="hash", numFeatures=1024).transform(sms)

# convert hashed symbols to TF-IDF
sms = IDF(inputCol="hash", outputCol="features").fit(sms).transform(sms)

sms.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,344,378,1006],[2.2391682769656747,2.892706319430574,3.684405173719015,4.244020961654438])|
|[dont, worry, guess, busy]      |(1024,[53,233,329,858],[4.618714411095849,3.557143394108088,4.618714411095849,4.937168142214383])   |
|[call, freephone]               |(1024,[138,396],[2.2391682769656747,3.3843005812686773])                                            |
|[win, cash, prize, prize, worth]|(1024,[31,69,387,428],[3.7897656893768414,7.284881949239966,4.4671645129686475,3.898659777615979])  |
+--------------------------------+--------------

### Training a spam classifier

In [17]:
# split the data into training and testing sets
sms_train, sms_test = sms.randomSplit([0.8, 0.2], seed=13)

# fit a logistic regresssion model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# make predictions on the testing data
predictions = logistic.transform(sms_test)

# create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1666|
|    0|       0.0| 2636|
|    1|       1.0| 3169|
|    0|       1.0| 2024|
+-----+----------+-----+



## Regression

Next you'll learn to create Linear Regression models. You'll also find out how to augment your data by engineering new predictors as well as a robiust approach to selecting ony the most relevant predictors.

### One-Hot Encoding

Dummy variables
- Each categorical level becomes a column
- Binary values indicate the presence (1) or absence (0) of the corresponding level
- Sparse reprersentation: store column index and value

One-hot encoding
- `from pyspark.ml.feature import OneHotEncoderEstimator`
- `onehot = OneHotEncoderEstimator(imputCols=['type_idx'], outputCols=['type_dummy'])`
- `onehot = onehot.fit(cars)`
- `onehot.categorySizes`
- `cars = onehot.transform(cars)`
- `cars.select('type, 'type_idx', 'type_dummy').distinct().sort('type_idx').show()`

Dense versus sparse
- `from pyspark.mllib.linalg import DenseVector, SparseVector`
- Store this vector:[1, 0, 0, 0, 0, 7, 0, 0])
- `DenseVector([1, 0, 0, 0, 0, 7, 0, 0])`
    - `out:` DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 7.0, 0.0, 0.0])
- `SparseVector(8, [0, 5], [1, 7])`
    - `out:` SparseVector(8, {0: 1.0, 5: 7.0})
- SparseVector(length of the vector, index of non-zero values, non-zero values)

In [18]:
# reset flights dataframe
flights = spark.read.csv('../data/flights.csv',
                        sep=',',
                        header=True,
                        inferSchema=True,
                        nullValue='NA')

flights = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights).transform(flights)

### Encoding flight origin

In [19]:
# import the one hot encoder class
from pyspark.ml.feature import OneHotEncoderEstimator

# create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights = onehot.transform(flights)

# check the results
flights.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SJC|    4.0|(7,[4],[1.0])|
|SMF|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



### Enocidng shirt sizes
You have data for a consignment of t-shirts. The data includes th size of the shirt, which is given as either S,M, L, or XL.

Here Are the counts for the different sizes.

`+----+-----+
|size|count|
+----+-----+
|   S|    8|
|   M|   15|
|   L|   20|
|  XL|    7|
+----+-----+`

The sizes are first converted to an index using `StringIndexer` and then one-hot encoded using `OneHotEncorderEstimator`

Output:

`+---+-------+-------------+
|size|   idx|    idx_dummy|
+---+-------+-------------+
|  S|    2.0|(3,[2],[1.0])|
|  M|    1.0|(3,[1],[1.0])|
|  L|    0.0|(3,[0],[1.0])|
| XL|    3.0|(3,[3],  [] )|
+---+-------+-------------+`

### Regression

Mininmizing Loss functiopn
- MSE = "Mean Squared Error"

Assemble predictors
- Predict `consumptiopn` using `mass`,`cyl`,and `type_dummy`.
- Consolidate predictors into a single column called `features`

Build regression model
- `from pyspark.ml.regression import LinearRegression`
- `regression = LinearRegression(labelCol='consumption')`
- `regression = regression.fit(cars_train)`
- `predictions = regression.transform(cars_test)`

Calculate RMSE
- `from pyspark.ml.evaluation import RegressionEvaluator`
- `RegressionEvaluator(labelCol='consumption').evaluate(predictions)`
- `RegressionEvaluator` can also calculate the following metrics
    - `mae` (Mean Absolute Error)
    - `r2` (R^2)
    - `mse` (Mean Squared Error)

Examine intercept
- `regression.intercept`
- This is the fuel consumption in the (hypothetical) case that:
    - `mass` = 0
    - `cyl` = 0

Examine Coefficients
- `regression.coefficients`

In [20]:
flights = flights.withColumn('km', flights.mile * 1.60934).drop('mile')

In [21]:
assembler = VectorAssembler(inputCols=['km'], outputCol='features')
flights = assembler.transform(flights)

flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=13)

In [22]:
flights.show(5)

+---+---+---+-------+------+---+------+--------+-----+-------+-------------+----------+------------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|org_idx|    org_dummy|        km|    features|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+----------+------------+
| 11| 20|  6|     US|    19|JFK|  9.48|     351| null|    2.0|(7,[2],[1.0])|3464.90902|[3464.90902]|
|  0| 22|  2|     UA|  1107|ORD| 16.33|      82|   30|    0.0|(7,[0],[1.0])| 508.55144| [508.55144]|
|  2| 20|  4|     UA|   226|SFO|  6.17|      82|   -8|    1.0|(7,[1],[1.0])| 542.34758| [542.34758]|
|  9| 13|  1|     AA|   419|ORD| 10.33|     195|   -5|    0.0|(7,[0],[1.0])|1989.14424|[1989.14424]|
|  4|  2|  5|     AA|   325|ORD|  8.92|      65| null|    0.0|(7,[0],[1.0])| 415.20972| [415.20972]|
+---+---+---+-------+------+---+------+--------+-----+-------+-------------+----------+------------+
only showing top 5 rows



### Flight duration model: Just distance

In [23]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

+--------+------------------+
|duration|prediction        |
+--------+------------------+
|385     |359.2696656028616 |
|135     |149.93838859748362|
|200     |224.09938202172748|
|64      |72.97656947741446 |
|259     |269.1561432154389 |
+--------+------------------+
only showing top 5 rows



17.039835602281954

### Interpreting the coefficients

In [24]:
# intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# coefficients
coefs = regression.coefficients
print(coefs)

# average minutes per km
minutes_per_km = coefs[0]

# average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

44.35943736789506
[0.07566768380408988]
792.9408828654666


### Flight duration model: Adding origin airport

In [25]:
flights = flights.drop('features')

# add org_dummy to features
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')
flights = assembler.transform(flights)

flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=13)

In [26]:
# create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# create predictions for the testing data
predictions = regression.transform(flights_test)

# calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

11.124011989821785

### Interpreting coefficients

The values for km and org_dummy have been assembled into features, which has eight columns with sparse representation. Column indices in features are as follows:

- 0 — km
- 1 — ORD
- 2 — SFO
- 3 — JFK
- 4 — LGA
- 5 — SMF
- 6 — SJC and
- 7 — TUS.

In [27]:
# average speed in km per hour
avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

# average minutes on ground at OGG
inter = regression.intercept
print(inter)

# average minutes on ground at JFK
avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

# average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

807.3233680861835
15.862995338536262
68.52994024026248
62.568008835633506


### Bucketing & Engineering
Bucketing
- Binning values
- `from pyspark.ml.feature import Bucketizer`
- `bucketizer = Bucketizer(splits=[3500, 4500, 6000, 6500], inputCol="rpm", outputCol= "rpm_bin")
- `cars = bucketizer.transform(cars)`
- onehotencode before applying to regression model

More feature engineering
- Operations on a single columns:
    - `log()`
    - `sqrt()`
    - `pow()`
- Operations on two columns:
    - product
    - ratio
    
Feature engineering examples:
- Mass & Height to BMI
    - potentially, BMI can be a more powerful predictor than mass or height in isolation
- Engineering density
    - cars = cars.withColumn('density_line', cars.mass / cars.length)    # Linear density
    - cars = cars.withColumn('density_quad', cars.mass / cars.length**2) # Area density
    - cars = cars.withColumn('density_cube', cars.mass / cars.length**3) # Volume density
<br> *note - powerful new features are often discovered through trial and error*


### Bucketing departure time

In [28]:
from pyspark.ml.feature import Bucketizer, OneHotEncoderEstimator

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[0.00, 03.00, 06.00, 09.00, 12.00, 15.00, 18.00, 21.00, 24.00], 
                     inputCol='depart', outputCol='depart_bucket')

# bucket the depatrue times
flights = buckets.transform(flights)
flights.select('depart', 'depart_bucket').show(5)

# create a one-hot encoder
onehot = OneHotEncoderEstimator(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed deptature times
flights = onehot.fit(flights).transform(flights)
flights.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  9.48|          3.0|
| 16.33|          5.0|
|  6.17|          2.0|
| 10.33|          3.0|
|  8.92|          2.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  9.48|          3.0|(7,[3],[1.0])|
| 16.33|          5.0|(7,[5],[1.0])|
|  6.17|          2.0|(7,[2],[1.0])|
| 10.33|          3.0|(7,[3],[1.0])|
|  8.92|          2.0|(7,[2],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



### Flight duration model: Adding departure time
`depart_dummy` index 8 to 14
- 8 — 00:00 - 03:00
- 9 — 03:00 - 06:00
- 10 — 06:00 - 09:00
- 11 — 09:00 - 12:00
- 12 — 12:00 - 15:00
- 13 — 15:00 - 18:00
- 14 — 18:00 - 21:00

In [29]:
# reset features columns to include depart_dummy
flights = flights.drop('features')
# create vectorassembler
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy'],
                            outputCol='features')
# create features column
flights = assembler.transform(flights)
# split data
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=13)
# create regression model and fit on train set
regression = LinearRegression(labelCol='duration').fit(flights_train)
# make predictions on test set
predictions = regression.transform(flights_test)

In [30]:
# find the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights depating between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[3] + regression.coefficients[8]
print(avg_night_jfk)

10.483676375513884
-4.121344067599443
47.572817783321035


### Regularization
Lasso
- absolute value of the coefficients
- can force coefficients to zero
- `lasso = LinearRegression(labelCol='consumption', elasticNetParam=1, regParam=0.1)`
Ridge
- square of the coefficients
- can force coefficients close to zero
- `ridge = LinearRegression(labelCol='consumption', elasticNetParam=0, regParam=0.1)`

### Flight duration model: More features!
These are the feature syou'll include in the next model:
- `km`
- `org` (origin airport, one-hot encoded, 8 levels)
- `depart` (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)
- `dow` (departure day of week, one-hot encoded, 7 levels)
- `mon` (departure month, one-hot encoded, 12 levels)

In [31]:
flights = flights.drop('features')

onehot = OneHotEncoderEstimator(inputCols=['dow', 'mon'], outputCols = ['dow_dummy', 'mon_dummy'])
flights = onehot.fit(flights).transform(flights)

assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy', 'dow_dummy', 'mon_dummy'],
                           outputCol='features')
flights = assembler.transform(flights)

flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=13)

In [32]:
# fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# make predictions on testing data
predictions = regression.transform(flights_test)

# calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.724055996754982
[0.07443672229195626,27.074222675233724,20.077136393625985,51.76577422204704,45.645021378984076,17.45535293535999,14.979185825703254,17.06442064280308,-15.113860995892185,1.6427071994710414,4.056829859905945,6.874198429259234,4.632713855387798,8.815481970316627,8.730109378588802,0.35363690972964,0.06173617510577866,-0.16088770207113512,0.19148688860628943,0.20333490710532326,0.12115889895844358,-2.1856825118460708,-2.288796502807716,-2.0823432825081207,-3.666577161918417,-4.156684025422529,-4.377352065170049,-4.528997974139636,-4.284372374700983,-3.985546221334758,-2.9283459130415714,-0.7856999249283695]


### Flight duration model: Regularisation!

In [33]:
# fit Lasso model ((α = 1)) to training data
lasso = LinearRegression(labelCol='duration', elasticNetParam=1, regParam=1).fit(flights_train)

# calculate the RMSE on testing data
lasso_rmse = RegressionEvaluator(labelCol='duration').evaluate(lasso.transform(flights_test))
print("The lasso test RSME is", lasso_rmse)

# look at the model coefficients
lasso_coeffs = lasso.coefficients
print(lasso_coeffs)

# number of zero coefficients
zero_coeff = sum([beta == 0 for beta in lasso.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The lasso test RSME is 11.70447567404799
[0.07352732182283632,5.363611378756951,0.0,29.02745297158733,21.883182046066523,0.0,-2.3369548237782367,0.0,0.0,0.0,0.0,0.0,0.0,1.1715298130259522,1.1899197632965113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Number of coefficients equal to 0: 25


## Ensembles & Pipelines
Finally you'll learn how to make your models more efficient. You'll find out how to use ipelines to make your code clearer and easier to maintain. Then you'll use cross-validation to better test your models and dselct good model parameters. Finally you'll dabble in two types of ensemble models.

### Pipeline
Streamline workflow and ensure that training and testing data are treated consistently and no leakage of information between these two sets takes place.

Leakage?
- `.fit()` to training data **only**
- `.transform` can be applied to both training and testing data

A pipeline consists of a series of operations. You could apply each operation individually.. or you could just apply the pipeline!

Cars model: Pipeline
- `from pyspark.ml import Pipeline`
- `pipeline = Pipeline(stages=[indexer, onehot, assemble, regression])`
- `pipeline = pipeline.fit(cars_train)`
- `predictions = pipeline.transform(cars_test)`

You can access individual stages using the `.stages` attribute
- `pipeline.stages[3] #fourth state = LinearRegression object`
- `print(pipeline.stages[3].intercept)`
- `print(pipeline.stages[3].coefficients)`

In [34]:
# create fresh flights DataFrame
flights = spark.read.csv('../data/flights.csv',
                        sep=',',
                        header=True,
                        inferSchema=True,
                        nullValue='NA')

flights = flights.withColumn('km', round(flights.mile * 1.60934))

### Flight duration model: Pipeline stages

In [35]:
# convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoderEstimator(inputCols=['org_idx', 'dow'], 
                                           outputCols=['org_dummy', 'dow_dummy'])

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'],
                           outputCol='features')
                                           
# a linear regression object
regression = LinearRegression(labelCol='duration')

### Flight duration model: Pipeline model

In [36]:
# split data
flights_train, flights_test = flights.randomSplit([0.8, 0.2], seed=13)

In [37]:
# import class fo creating pipeline
from pyspark.ml import Pipeline

# construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

### SMS spam pipeline

In [38]:
# break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# remove stop words
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='terms')

# apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol=remover.getOutputCol(), outputCol='hash')
idf = IDF(inputCol=hasher.getOutputCol(), outputCol='features')

# create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
sms_pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

### Cross-Validation
Grid and cross-validator
- `from pyspark.ml.tuning import CrossValidator, ParamGridBuilder`
- `params = ParamGridBuilder().build()`
- `cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=10, seed=13)`
- `cv = cv.fit(cars_train`
- `cv.avgMetrics` average RMSE
- `evaluator.evaluate(cv.transform(cars_test))`

### Cross validating simple flight duration
In this exercise yoiu're going to train a simple model for flight duration using cross-validation. Travel time is usually strongly correlated with distance, so using the `km` column alone should give a decent model

In [39]:
flights = VectorAssembler(inputCols=['km'], outputCol='features').transform(flights)

flights_train, flights_test = flights.limit(1232).randomSplit([0.8, 0.2], seed=13)

In [40]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# create an empty parameter grid
params = ParamGridBuilder().build()

# create object for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# create a cross validator
cv = CrossValidator(estimator=regression,
                   estimatorParamMaps=params,
                   evaluator=evaluator,
                   numFolds=5,
                   seed=13)

# train and test model on multiple folds of the training data
cv = cv.fit(flights_train)

### Cross validating flight duration model pipeline
In this exercise you'll add the `org` field to the model However, since `org` is categorical, there's more work to be done before itr can be included; it must first be transofrmed to an index and then one-hot encoded before being assembled with `km` and used to build the regression model. We'll wrap these operations up in a pipeline.

In [41]:
flights = flights.drop('features')

In [42]:
# create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# create a one-hot encoder for the index org field
onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# create a pipeline and cross-validator
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
                   estimatorParamMaps=params,
                   evaluator=evaluator)

### Grid Search
Parameter grid
- `from pyspark.ml.tuning import ParamGridBuilder`
- `params = ParamGridBuilder()`
- `params = params.addGrid(regression.fitIntercept, [True, False])` add grid points
- `params = params.build()` construct the grid

Grid search with cross-validation
- `cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator)`
- `cv = cv.setNumFolds(10).setSeed(13).fit(cars_train)`
<br>*since there are 2 points on the grid and 10 folds; this translates to 20 models*
- `cv.avgMetrics`
<br> > `[0.800663722151, 0.9079977823]` #in a list because you get one value for each point on the grid
- `cv.bestModel` # access the best model
- `predictions = cv.transform(cars_test)` # use the cross-validator object it will use the best model automatically
- `cv.bestModel.explainParam('fitIntercept')` # retrieve the best parameter

### Optimizing flights linear regression

In [43]:
# create parameter grid
params = ParamGridBuilder()

# add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]).addGrid(regression.elasticNetParam, [0.0, 0.5, 1,0])

# build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

Number of models to be tested:  16


In [44]:
# create cross-validator
cv = CrossValidator(estimator=pipeline,
                   estimatorParamMaps=params,
                   evaluator=evaluator,
                   numFolds=5)

### Dissecting the best flight duration model

In [47]:
flights_train, flights_test = flights.limit(1232).randomSplit([0.8, 0.2], seed=13)

cv = cv.fit(flights_train)

# get the best model from cross validation
best_model = cv.bestModel

# look at the stages in the best model
print(best_model.stages)

# get the parameters fr the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# generate predicrtions on testing darta using the best model then calculate RMSE
predictions = best_model.transform(flights_test)
evaluator.evaluate(predictions)

AttributeError: 'CrossValidator' object has no attribute 'bestModel'

### SMS spam optimised

In [58]:
# create parameter grid
params = ParamGridBuilder()

# add grid for hashin trick parameters
params = params.addGrid(hasher.numFeatures, [1024, 4096, 16384]).addGrid(hasher.binary, [True, False])

# add grid for logistic regression parameters
params = params.addGrid(logistic.regParam, [0.01, 0.1, 1.0, 10.0]).addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0])

params = params.build()

### Ensemble
- It's a collection of models
- Wisdom of the Crowd - collective opinion of a group is better than that of a single expert
> **Diversity** and **independence** are important because the best collective decisions are the product of disagreement and contest, not consensus or compromise. 
<br> - James Surowiecki, *The Wisdom of Crowds*

Random Forest
- works in parallel
- an ensemble of Decision Trees
- Creating model diversity:
    - each tree trained on *random subset* of data
    - *random subset* of features used for splitting at each node
- No two trees in the forest should be the same
- `from pyspark.ml.classification import RandomForestClassifier`
- `forest = RandomForestClassifier(numTrees=5)`
- `forest = forest.fit(cars_train)`
- `forest.trees`
- `forest.featureImportances`

Gradient-Boosted Trees
-  works in series
- Iterative boosting algorithm:
    1. Build a Decision Tree and add to ensemble
    2. Predict label for each training instance using ensemble.
    3. Compare predictions with known labels.
    4. Emphasize training instances with incorrecrt predictions.
    5. Return to 1.
- Model improves each iteration
- `from pyspark.ml.classification import GBTClassifier`
- `gbt=GBTClassifier(maxIter=20)`
- `gbt = gbt.fit(cars_train)`

### Delayed flights with Gradient-Boosted Trees

In [62]:
# prepare data
flights_train, flights_test = flights_assembled.select('mon', 'depart', 'duration', 'features', 'label').randomSplit([0.8, 0.2], seed=13)

In [64]:
# Import the classes required
from pyspark.ml.classification import GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# create model objects and train on training data
tree = DecisionTreeClassifier().fit(flights_train)
gbt = GBTClassifier().fit(flights_train)

# Compare AUC on testing data
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(tree.transform(flights_test))
evaluator.evaluate(gbt.transform(flights_test))

# find the number of trees and the relative iportance of features
print(gbt.trees)
print(gbt.featureImportances)

[DecisionTreeRegressionModel (uid=dtr_a825bf63fc04) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_86d1edf273db) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_9ccd639df39e) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_1759dcf98e9b) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_700af3baefa9) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_d36e2fa87dce) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_946ba3bba509) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_50be7f809177) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_419e2067639e) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_d79d007a77a7) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_116b9ecfc139) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_4ce1dbdce04f) of depth 5 with 63 nodes, DecisionTreeRegressionModel (uid=dtr_e70f91999e4d) of depth 5 with 63 nodes

### Delayed flights with a Random Forest

In [65]:
from pyspark.ml.classification import RandomForestClassifier

# create a random forest classifier
forest = RandomForestClassifier()

# create a parameter grid
params = ParamGridBuilder().addGrid(
    forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']).addGrid(
    forest.maxDepth, [2, 5, 10]).build()

# create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()

# create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

### Evaluating Random Forest

In [67]:
cv = cv.fit(flights_train)

In [68]:
# Average AUC for each paramter combination in grid
avg_auc = cv.avgMetrics

# average AUC for the best model
best_model_auc = max(avg_auc)

# whats the optiml parameter value?
opt_max_depth = cv.bestModel.explainParam('maxDepth')
opt_feat_substrat = cv.bestModel.explainParam('featureSubsetStrategy')

# AUC for best model on testing data
best_auc = evaluator.evaluate(cv.transform(flights_test))

In [69]:
print(opt_max_depth)
print(opt_feat_substrat)
print(best_auc)

maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5, current: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto, current: all)
0.7242267382150237


In [70]:
spark.stop()